In [3]:
from bs4 import BeautifulSoup
import requests

import re

import numpy as np
import pandas as pd

import base64
from requests import post, get

import json
import time

from datetime import datetime

import warnings

# Suppress all warnings
warnings.filterwarnings("ignore")

StatementMeta(sparkpool, 6, 2, Finished, Available)

# **1. Data Scraping**

## **1.1. kworb.net Scraping**

In [51]:
col_title = ['Song_ID', 'Lead_Artist_ID', 'Daily_Stream', 'Total_Stream', 'Extract_Date']
top_200_track_daily = pd.DataFrame(columns = col_title)

url = 'https://kworb.net/spotify/country/global_daily.html'
page = requests.get(url)
soup = BeautifulSoup(page.text, 'html')
col_data = soup.find_all('tr')

for row in col_data[1:]:
    row_data = row.find_all('td')

    track_url = row_data[2].find_all('a')[1].get('href')
    artist_url = row_data[2].find_all('a')[0].get('href')
    daily_stream = row_data[-5].text.strip()
    total_stream = row_data[-1].text.strip()

    track_pattern = r'../track/([^_]+)\.html'
    track_match = re.search(track_pattern, track_url)
    if track_match:
        track_id = track_match.group(1)
    else:
        track_id = np.NaN

    artist_pattern = r'../artist/([^_]+)\.html'
    artist_match = re.search(artist_pattern, artist_url)
    if artist_match:
        artist_id = artist_match.group(1)
    else:
        artist_id = np.NaN

    now = datetime.now()
    extract_date = now.strftime("%Y-%m-%d %H:%M:%S")

    individual_row_data = [track_id, artist_id, daily_stream, total_stream, extract_date]

    length = len(top_200_track_daily)
    top_200_track_daily.loc[length] = individual_row_data

top_200_track_daily['Daily_Stream'] = top_200_track_daily['Daily_Stream'].apply(lambda x: x.replace(",",""))
top_200_track_daily['Daily_Stream'] = top_200_track_daily['Daily_Stream'].astype(int)
top_200_track_daily['Total_Stream'] = top_200_track_daily['Total_Stream'].apply(lambda x: x.replace(",",""))
top_200_track_daily['Total_Stream'] = top_200_track_daily['Total_Stream'].astype(int)
top_200_track_daily['Extract_Date'] = pd.to_datetime(top_200_track_daily['Extract_Date'])
top_200_track_daily

StatementMeta(sparkpool, 6, 50, Finished, Available)

,Song_ID,Lead_Artist_ID,Daily_Stream,Total_Stream,Extract_Date
0,2qSkIjg1o9h3YT9RAgYN75,74KM79TiuVKeVCqs8QtB0B,9845584,692939277,2024-06-30 01:05:09
1,5N3hjp1WNayUPZrA8kJmJP,74KM79TiuVKeVCqs8QtB0B,9776098,232231734,2024-06-30 01:05:09
2,6dOtVTDdiauQNBQEDOtlAB,6qqNVTkY8uBg9cP3Jd7DAH,8619920,343601388,2024-06-30 01:05:09
3,1kPhV0KQui1phEpjnWIqUN,1oSPZhvZMIrWW5I41kPkkY,6625387,6625387,2024-06-30 01:05:09
4,6XjDF6nds4DE2BBbagZol6,7CvTknweLr9feJtRGrpDBy,6385692,743197603,2024-06-30 01:05:09
...,...,...,...,...,...
195,3AVrVz5rK8Hrqo9YGiVGN5,1QAJqy2dA3ihHBFIHRphZj,1326745,877612250,2024-06-30 01:05:09
196,5496lks6V3fh1mZAB6IAap,5MraexJKZDrQYzS98kNwie,1322658,1322658,2024-06-30 01:05:09
197,4y1LsJpmMti1PfRQV9AWWe,2BTZIqw0ntH9MvilQ3ewNY,1319085,3636855,2024-06-30 01:05:09
198,3yfqSUWxFvZELEM4PmlwIR,7dGJo4pcD2V6oG8kP0tJRR,1317100,878523205,2024-06-30 01:05:09


### **1.1.1. Artist IDs**

In [5]:
url = 'https://kworb.net/spotify/artists.html'
page = requests.get(url)
soup = BeautifulSoup(page.text, 'html')
col_data = soup.find_all('tr')

artist_ids = []
for row in col_data[1:]:
    row_data = row.find_all('td')
    artist_url = row_data[0].find_all('a')[0].get('href')
    pattern = r'/spotify/artist/([^_]+)_songs\.html'
    match = re.search(pattern, artist_url)
    if match:
        artist_id = match.group(1)
    else:
        artist_id = np.NaN

    artist_ids.append(artist_id)

StatementMeta(sparkpool, 6, 4, Finished, Available)

In [6]:
# Top n artists
n = 500

artist_ids = artist_ids[0:n]
for artist_id in set(top_200_track_daily['Lead_Artist_ID']):
    if artist_id not in artist_ids:
        artist_ids.append(artist_id)
len(artist_ids)

StatementMeta(sparkpool, 6, 5, Finished, Available)

551

### **1.1.2. Song Stream Table**

In [7]:
col_title = ['Song_ID', 'Stream', 'Extract_Date']
song_stream = pd.DataFrame(columns = col_title)

for artist_id in artist_ids:
    pattern_url = f'https://kworb.net/spotify/artist/{artist_id}_songs.html'

    page = requests.get(pattern_url)
    soup = BeautifulSoup(page.text, 'html')

    col_data = soup.find_all('tr')

    for row in col_data[5:]:
        row_data = row.find_all('td')
        song_url = row_data[0].find_all('a')[0].get('href')
        pattern = r'https://open.spotify.com/track/([^_]+)'
        match = re.search(pattern, song_url)

        song_id = match.group(1)
        streams = [data.text.strip() for data in row_data][1]
        
        now = datetime.now()
        extract_date = now.strftime("%Y-%m-%d %H:%M:%S")

        individual_row_data = [song_id, streams, extract_date]

        length = len(song_stream)
        song_stream.loc[length] = individual_row_data

StatementMeta(sparkpool, 6, 6, Finished, Available)

In [8]:
song_stream['Stream'] = song_stream['Stream'].apply(lambda x: x.replace(",",""))
song_stream['Stream'] = song_stream['Stream'].astype(int)
song_stream['Extract_Date'] = pd.to_datetime(song_stream['Extract_Date'], format='%Y-%m-%d %H:%M:%S')
song_stream.drop_duplicates(subset='Song_ID', keep='first', inplace=True)

StatementMeta(sparkpool, 6, 7, Finished, Available)

## **1.2. Spotify Web API**

### **1.2.1. Token and Authorization**

In [9]:
client_id = 'ae28bbc5ae7f4292be5c065babb01997'
client_secret = '0a4f4c9a2e7147dc8d31ac6914f50132'

def get_token():
    url = 'https://accounts.spotify.com/api/token'
    
    auth_string = client_id + ':' + client_secret
    auth_bytes = auth_string.encode("utf-8")
    auth_base64 = str(base64.b64encode(auth_bytes), "utf-8")

    headers = {
        'content-type': 'application/x-www-form-urlencoded',
        'Authorization': 'Basic ' + auth_base64
    }

    form = {
        'grant_type': 'client_credentials'
    }

    result = post(url, headers=headers, data=form)
    json_result = json.loads(result.content)
    token = json_result['access_token']
    return token

def get_auth_header(token):
    return {"Authorization": f"Bearer {token}"}

StatementMeta(sparkpool, 6, 8, Finished, Available)

### **1.2.2. Artist Tables**

In [10]:
def search_for_artists(artist_ids, artist_info_list, artist_follower_list, artist_genres_list, max_retries=5, backoff_factor=1.0):
    url = f"https://api.spotify.com/v1/artists?ids={artist_ids}"
    token = get_token()
    headers = get_auth_header(token)

    for attempt in range(max_retries):
        response = get(url, headers=headers)
        
        if response.status_code == 200:
            
            json_result = response.json()  # use .json() method directly
            artists = pd.json_normalize(json_result['artists'])
            
            #artist_info table
            artist_info = artists[['id', 'name', 'images']]
            artist_info_list.append(artist_info)

            #artist_follower table
            artist_follower = artists[['id', 'followers.total']]
            now = datetime.now()
            artist_follower['Extract_Date'] = now.strftime("%Y-%m-%d %H:%M:%S")
            artist_follower_list.append(artist_follower)
            
            #artist_genres table
            artist_genres = artists[['id', 'genres']]
            artist_genres_list.append(artist_genres)

            return artist_info_list, artist_follower_list, artist_genres_list
        
        elif response.status_code == 429:
            retry_after = int(response.headers.get("Retry-After", backoff_factor * (2 ** attempt)))
            print(f"Too many requests. Retrying in {retry_after} seconds...")
            time.sleep(retry_after)
        
        else:
            response.raise_for_status()
    
    raise Exception("Maximum retries exceeded")

StatementMeta(sparkpool, 6, 9, Finished, Available)

In [11]:
concat_artist_ids = ''
count = 1

artist_info_list = []
artist_follower_list = []
artist_genres_list = []
batch = 50

for artist_id in artist_ids:
    if count % batch in range(1,batch):
        concat_artist_ids = concat_artist_ids + ',' + str(artist_id)
        if count == len(artist_ids):
            concat_artist_ids = concat_artist_ids[1:]
            try:
                search_for_artists(concat_artist_ids, artist_info_list, artist_follower_list, artist_genres_list)
            except:
                continue
    else:
        concat_artist_ids = concat_artist_ids + ',' + str(artist_id)
        concat_artist_ids = concat_artist_ids[1:]
        try:
            search_for_artists(concat_artist_ids, artist_info_list, artist_follower_list, artist_genres_list)
        except:
            continue
        
        concat_artist_ids = ''

    count += 1

StatementMeta(sparkpool, 6, 10, Finished, Available)

In [12]:
artist_info = pd.concat(artist_info_list, ignore_index=True)
artist_follower = pd.concat(artist_follower_list, ignore_index=True)
artist_genres = pd.concat(artist_genres_list, ignore_index=True)

StatementMeta(sparkpool, 6, 11, Finished, Available)

#### **1.2.2.1. Artist Info Table**

In [13]:
artist_info.dropna(inplace=True)
artist_info.drop(artist_info[artist_info['images'].apply(lambda x: len(x) == 0)].index,
                 axis=0, inplace=True)
artist_info['images'] = artist_info['images'].apply(lambda x: x[0]['url'])
artist_info.rename(columns={'id': 'Artist_ID',
                            'name': 'Artist_Name',
                            'images': 'Artist_Image'},
                            inplace=True)

StatementMeta(sparkpool, 6, 12, Finished, Available)

#### **1.2.2.2. Artist Follower Table**

In [14]:
artist_follower.dropna(inplace=True)
artist_follower['followers.total'] = artist_follower['followers.total'].astype(int)
artist_follower.rename(columns={'id': 'Artist_ID',
                                'followers.total': 'Followers'},
                                inplace=True)
artist_follower['Extract_Date'] = pd.to_datetime(artist_follower['Extract_Date'], format='%Y-%m-%d %H:%M:%S')
artist_follower = pd.merge(artist_info, artist_follower, on='Artist_ID', how='inner')[['Artist_ID', 'Followers', 'Extract_Date']]

StatementMeta(sparkpool, 6, 13, Finished, Available)

#### **1.2.2.3. Artist - Genre Table**

In [15]:
artist_genres = artist_genres.explode('genres')
artist_genres.dropna(inplace=True)

all_genres = [genre for genre in artist_genres['genres'] if genre is not np.NaN]
unique_genres = list(set(all_genres))

genre_to_code = {genre: code + 1 for code, genre in enumerate(unique_genres)}
artist_genres['genres'] = artist_genres['genres'].apply(lambda genre: genre_to_code[genre])
artist_genres = artist_genres.reset_index(drop=True)
artist_genres.rename(columns={'genres': 'Genre_ID',
                              'id': 'Artist_ID'},
                              inplace=True)

no_artist = []
for artist in artist_genres['Artist_ID'].tolist():
    if artist not in artist_info['Artist_ID'].tolist():
        no_artist.append(artist)

artist_genres.drop(artist_genres[artist_genres['Artist_ID'].isin(no_artist)].index, axis=0, inplace=True)

StatementMeta(sparkpool, 6, 14, Finished, Available)

#### **1.2.2.4. Genres Table**

In [16]:
genres = pd.DataFrame(list(genre_to_code.items()), columns=['Genre_Name', 'Genre_ID'])[['Genre_ID', 'Genre_Name']]

StatementMeta(sparkpool, 6, 15, Finished, Available)

### **1.2.3. Song Tables**

In [17]:
def search_for_tracks(track_ids, song_info_list, artist_song_list, album_song_list, max_retries=5, backoff_factor=1.0):
    url = f"https://api.spotify.com/v1/tracks?ids={track_ids}"
    token = get_token()
    headers = get_auth_header(token)

    for attempt in range(max_retries):
        response = get(url, headers=headers)
        
        if response.status_code == 200:
            
            json_result = response.json()  # use .json() method directly
            songs = pd.json_normalize(json_result['tracks'])
            
            #song_info table
            song_info = songs[['id', 'name', 'explicit', 'duration_ms', 'track_number']]
            song_info_list.append(song_info)

            #artist_song table
            artist_song = songs[['id', 'artists', 'album.artists']]
            artist_song_list.append(artist_song)
            
            #album_song table
            album_song = songs[['id', 'album.id']]
            album_song_list.append(album_song)

            return song_info_list, artist_song_list, album_song_list
        
        elif response.status_code == 429:
            retry_after = int(response.headers.get("Retry-After", backoff_factor * (2 ** attempt)))
            print(f"Too many requests. Retrying in {retry_after} seconds...")
            time.sleep(retry_after)
        
        else:
            response.raise_for_status()
    
    raise Exception("Maximum retries exceeded")

StatementMeta(sparkpool, 6, 16, Finished, Available)

In [18]:
concat_tracks_ids = ''
count = 1

song_info_list = []
artist_song_list = []
album_song_list = []
batch = 50

for track_id in song_stream['Song_ID']:
    if count % batch in range(1,batch):
        concat_tracks_ids = concat_tracks_ids + ',' + str(track_id)
        if count == len(song_stream['Song_ID']):
            concat_tracks_ids = concat_tracks_ids[1:]
            search_for_tracks(concat_tracks_ids, song_info_list, artist_song_list, album_song_list)
    else:
        concat_tracks_ids = concat_tracks_ids + ',' + str(track_id)
        concat_tracks_ids = concat_tracks_ids[1:]
        search_for_tracks(concat_tracks_ids, song_info_list, artist_song_list, album_song_list)
        
        concat_tracks_ids = ''

    count += 1

StatementMeta(sparkpool, 6, 17, Finished, Available)

In [19]:
song_info = pd.concat(song_info_list, ignore_index=True)
artist_song = pd.concat(artist_song_list, ignore_index=True)
album_song = pd.concat(album_song_list, ignore_index=True)

StatementMeta(sparkpool, 6, 18, Finished, Available)

#### **1.2.3.1. Song Info Table**

In [20]:
song_info.drop_duplicates(inplace=True)

song_info.rename(columns={'id': 'Song_ID',
                           'name': 'Song_Name',
                           'explicit': 'Explicit',
                           'duration_ms': 'Duration',
                           'track_number': 'Track_Number'},
                           inplace=True)

StatementMeta(sparkpool, 6, 19, Finished, Available)

#### **1.2.3.2. Lead Artist Table**

In [21]:
def extract_ids(artists):
    return [artist['id'] for artist in artists]

artist_song['artists'] = artist_song['artists'].apply(extract_ids)
artist_song['album.artists'] = artist_song['album.artists'].apply(extract_ids)

def find_lead_artists(row):
    return [artist for artist in row['artists'] if artist in row['album.artists']]

artist_song['lead_artist'] = artist_song.apply(find_lead_artists, axis=1)

def find_feature_artists(row):
    return [artist for artist in row['artists'] if artist not in row['album.artists']]

artist_song['feature_artist'] = artist_song.apply(find_feature_artists, axis=1)

lead_artist = artist_song[['lead_artist', 'id']]
lead_artist.rename(columns={'lead_artist': 'Lead_Artist_ID','id': 'Song_ID'}, inplace=True)
lead_artist = lead_artist.explode('Lead_Artist_ID')
lead_artist.drop_duplicates(inplace=True)

merge_df = pd.merge(lead_artist, artist_info, left_on='Lead_Artist_ID',
                    right_on='Artist_ID', how='left')
no_lead_artist_rows = merge_df[merge_df['Lead_Artist_ID'].isnull()]['Song_ID'].tolist()
artist_song['lead_artist'] = artist_song.apply(lambda x: x['artists'] if x['id'] in no_lead_artist_rows
                                               else x['lead_artist'], axis=1)
artist_song['feature_artist'] = artist_song.apply(lambda x: [] if x['id'] in no_lead_artist_rows
                                                  else x['feature_artist'], axis=1)
lead_artist = artist_song[['lead_artist', 'id']]
lead_artist.rename(columns={'lead_artist': 'Lead_Artist_ID','id': 'Song_ID'}, inplace=True)
lead_artist = lead_artist.explode('Lead_Artist_ID')
lead_artist.drop_duplicates(inplace=True)

merge_df = pd.merge(lead_artist, artist_info, left_on='Lead_Artist_ID',
                    right_on='Artist_ID', how='left')
merge_df.dropna(inplace=True)

lead_artist = merge_df[['Lead_Artist_ID', 'Song_ID']]

StatementMeta(sparkpool, 6, 20, Finished, Available)

#### **1.2.3.3. Feature Artist Table**

In [22]:
feature_artist = artist_song[['feature_artist', 'id']]
feature_artist.rename(columns={'feature_artist': 'Feature_Artist_ID','id': 'Song_ID'}, inplace=True)
feature_artist = feature_artist.explode('Feature_Artist_ID')
feature_artist.drop_duplicates(inplace=True)

merge_df = pd.merge(feature_artist, artist_info, left_on='Feature_Artist_ID',
                    right_on='Artist_ID', how='left')
merge_df.dropna(inplace=True)

feature_artist = merge_df[['Feature_Artist_ID', 'Song_ID']]

StatementMeta(sparkpool, 6, 21, Finished, Available)

### **1.2.4. Album Tables**

In [23]:
def search_for_albums(album_ids, album_info_list, artist_album_list, max_retries=5, backoff_factor=1.0):
    url = f"https://api.spotify.com/v1/albums?ids={album_ids}"
    token = get_token()
    headers = get_auth_header(token)

    for attempt in range(max_retries):
        response = get(url, headers=headers)
        
        if response.status_code == 200:
            
            json_result = response.json()  # use .json() method directly
            albums = pd.json_normalize(json_result['albums'])
            
            #album_info table
            album_info = albums[['id', 'name', 'total_tracks', 'album_type',
                                 'release_date', 'images', 'label']]
            album_info_list.append(album_info)

            #artist_album table
            artist_album = albums[['id', 'artists']]
            artist_album_list.append(artist_album)

            return album_info_list, artist_album_list
        
        elif response.status_code == 429:
            retry_after = int(response.headers.get("Retry-After", backoff_factor * (2 ** attempt)))
            print(f"Too many requests. Retrying in {retry_after} seconds...")
            time.sleep(retry_after)
        
        else:
            response.raise_for_status()
    
    raise Exception("Maximum retries exceeded")

StatementMeta(sparkpool, 6, 22, Finished, Available)

In [25]:
album_list = list(set(album_song['album.id']))

concat_albums_ids = ''
count = 1

album_info_list = []
artist_album_list = []

for album_id in album_list:
    if count % 20 in range(1,20):
        concat_albums_ids = concat_albums_ids + ',' + str(album_id)
        if count == len(album_list):
            concat_albums_ids = concat_albums_ids[1:]
            search_for_albums(concat_albums_ids, album_info_list, artist_album_list)
    else:
        concat_albums_ids = concat_albums_ids + ',' + str(album_id)
        concat_albums_ids = concat_albums_ids[1:]
        search_for_albums(concat_albums_ids, album_info_list, artist_album_list)
        
        concat_albums_ids = ''

    count += 1

StatementMeta(sparkpool, 6, 24, Finished, Available)

In [26]:
album_info = pd.concat(album_info_list, ignore_index=True)
artist_album = pd.concat(artist_album_list, ignore_index=True)

StatementMeta(sparkpool, 6, 25, Finished, Available)

#### **1.2.4.1. Album Info Table**

In [27]:
album_info.drop(album_info[album_info['images'].apply(lambda x: len(x) == 0)].index,
                axis=0, inplace=True)
album_info['images'] = album_info['images'].apply(lambda x: x[0]['url'])

album_info.rename(columns={'id': 'Album_ID',
                           'name': 'Album_Name',
                           'total_tracks': 'Total_Tracks',
                           'album_type': 'Album_Type',
                           'release_date': 'Release_Date',
                           'images': 'Album_Image',
                           'label': 'Label'},
                           inplace=True)

album_info['Release_Date'] = pd.to_datetime(album_info['Release_Date'], format='%Y-%m-%d')

StatementMeta(sparkpool, 6, 26, Finished, Available)

#### **1.2.4.2. Album Song Table**

In [28]:
album_song.drop_duplicates(inplace=True)
album_song.rename(columns={'album.id': 'Album_ID',
                           'id': 'Song_ID'}, inplace=True)

no_album = []
for album in album_song['Album_ID'].tolist():
    if album not in album_info['Album_ID'].tolist():
        no_album.append(album)

album_song.drop(album_song[album_song['Album_ID'].isin(no_album)].index,
                axis=0, inplace=True)

StatementMeta(sparkpool, 6, 27, Finished, Available)

#### **1.2.4.3. Artist - Album Table**

In [29]:
artist_album['artists'] = artist_album['artists'].apply(extract_ids)
artist_album = artist_album.explode('artists')
artist_album.drop_duplicates(inplace=True)
artist_album.rename(columns={'id': 'Album_ID',
                             'artists': 'Artist_ID'}, inplace=True)

StatementMeta(sparkpool, 6, 28, Finished, Available)

In [30]:
no_album = []
for album in artist_album['Album_ID'].tolist():
    if album not in album_info['Album_ID'].tolist():
        no_album.append(album)

StatementMeta(sparkpool, 6, 29, Finished, Available)

In [31]:
no_artist = []
for artist in artist_album['Artist_ID'].tolist():
    if artist not in artist_info['Artist_ID'].tolist():
        no_artist.append(artist)

StatementMeta(sparkpool, 6, 30, Finished, Available)

In [32]:
drop_rows_based_album = artist_album[artist_album['Album_ID'].isin(list(set(no_album)))].index.to_list()
drop_rows_based_artist = artist_album[artist_album['Artist_ID'].isin(list(set(no_artist)))].index.to_list()
artist_album.drop(drop_rows_based_album, axis=0, inplace=True)
artist_album.drop(drop_rows_based_artist, axis=0, inplace=True)

StatementMeta(sparkpool, 6, 31, Finished, Available)

# **2. Exporting Data**

In [52]:
file_system_name = 'datasets'
storage_account_name = 'spotifyprojectadls'
adls_path = f"abfss://{file_system_name}@{storage_account_name}.dfs.core.windows.net/"

top_200_track_daily.to_parquet(adls_path + 'top_200_track_daily.parquet', index=False)
song_stream.to_parquet(adls_path + 'song_stream.parquet', index=False)
artist_info.to_parquet(adls_path + 'artist_info.parquet', index=False)
artist_follower.to_parquet(adls_path + 'artist_follower.parquet', index=False)
artist_genres.to_parquet(adls_path + 'artist_genres.parquet', index=False)
genres.to_parquet(adls_path + 'genres.parquet', index=False)
lead_artist.to_parquet(adls_path + 'lead_artist.parquet', index=False)
feature_artist.to_parquet(adls_path + 'feature_artist.parquet', index=False)
album_song.to_parquet(adls_path + 'album_song.parquet', index=False)
artist_album.to_parquet(adls_path + 'artist_album.parquet', index=False)
album_info.to_parquet(adls_path + 'album_info.parquet', index=False)
song_info.to_parquet(adls_path + 'song_info.parquet', index=False)

StatementMeta(sparkpool, 6, 51, Finished, Available)